# Workshop 03: Machine Learning Models for Industrial Safety
This notebook loads the Kaggle dataset 'Industrial Safety and Health Analytics Database', performs EDA (Exploratory Data Analysis) and Data Quality checks, and applies two Supervised Machine Learning algorithms (Random Forest and Logistic Regression) to predict the accident level.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
import warnings

warnings.filterwarnings('ignore')
sns.set_theme(style="whitegrid")

## 1. Carga de Datos
We load the dataset using `kagglehub`.

In [ ]:
# Install kagglehub if not present
!pip install -q kagglehub
import kagglehub
import os

# Download latest version of the dataset
path = kagglehub.dataset_download("ihmstefanini/industrial-safety-and-health-analytics-database")
print("Path to dataset files:", path)

# Load the dataset
csv_file = os.path.join(path, "IHMStefanini_industrial_safety_and_health_database.csv")
df = pd.read_csv(csv_file)
df.head()

## 2. Calidad de data (Nulos)
Verificando la calidad de data. Determinando si hay nulos en nuestro dataset.

In [ ]:
print("Valores nulos por columna:")
print(df.isnull().sum())

# Eliminar filas con nulos en variables clave (si los hubiera)
features = ['Countries', 'Local', 'Industry Sector', 'Genre', 'Employee ou Terceiro', 'Risco Critico']
target = 'Accident Level'
df = df.dropna(subset=features + [target])

## 3. Analizando datos por variable (Univariado)
Visualizando la distribución de las variables más importantes de forma individual.

In [ ]:
plt.figure(figsize=(15, 10))

plt.subplot(2, 2, 1)
sns.countplot(y="Industry Sector", data=df, order=df['Industry Sector'].value_counts().index)
plt.title("Distribución por Sector Industrial")

plt.subplot(2, 2, 2)
sns.countplot(x="Accident Level", data=df, order=['I', 'II', 'III', 'IV', 'V', 'VI'], palette="Reds")
plt.title("Distribución del Nivel de Accidente (Target)")

plt.subplot(2, 2, 3)
sns.countplot(x="Genre", data=df, palette="pastel")
plt.title("Distribución por Género")

plt.subplot(2, 2, 4)
sns.countplot(x="Employee ou Terceiro", data=df, palette="Set2")
plt.title("Distribución por Tipo de Empleado")

plt.tight_layout()
plt.show()

## 4. Análisis de variables en conjunto (Bivariado)
Relacionando las variables independientes con nuestra variable objetivo (`Accident Level`).

In [ ]:
plt.figure(figsize=(12, 6))
sns.countplot(x="Industry Sector", hue="Accident Level", data=df, 
              hue_order=['I', 'II', 'III', 'IV', 'V', 'VI'], palette="Reds")
plt.title("Nivel de Accidente según el Sector Industrial")
plt.xticks(rotation=45)
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
sns.countplot(x="Employee ou Terceiro", hue="Accident Level", data=df, 
              hue_order=['I', 'II', 'III', 'IV', 'V', 'VI'], palette="Reds")
plt.title("Nivel de Accidente según el Tipo de Empleado (Propio vs Tercero)")
plt.show()

## 5. Pre-procesamiento de Datos
Preparar los datos para los modelos de Machine Learning (Supervisado).

In [ ]:
X = df[features]
y = df[target]

# Dividir los datos (80% entrenamiento, 20% prueba)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"Dimensiones de entrenamiento: {X_train.shape}")
print(f"Dimensiones de prueba: {X_test.shape}")

# Transformador para convertir variables de texto a números (One-Hot Encoding)
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), features)
    ])

## 6. Aplicando Algoritmos (Machine Learning Supervisado)
Vamos a aplicar los dos algoritmos elegidos: Random Forest y Logistic Regression.

In [ ]:
# --- Modelo 1: Bosques Aleatorios (Random Forest) ---
rf_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42, n_estimators=100))
])

rf_pipeline.fit(X_train, y_train)
rf_predictions = rf_pipeline.predict(X_test)

print("--- Resultados de Random Forest ---")
print(f"Precisión (Accuracy): {accuracy_score(y_test, rf_predictions):.4f}")
print("Reporte de Clasificación:")
print(classification_report(y_test, rf_predictions, zero_division=0))

In [ ]:
# --- Modelo 2: Regresión Logística Multinomial ---
lr_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(random_state=42, max_iter=1000, multi_class='multinomial'))
])

lr_pipeline.fit(X_train, y_train)
lr_predictions = lr_pipeline.predict(X_test)

print("--- Resultados de Regresión Logística ---")
print(f"Precisión (Accuracy): {accuracy_score(y_test, lr_predictions):.4f}")
print("Reporte de Clasificación:")
print(classification_report(y_test, lr_predictions, zero_division=0))

## 7. Asignando modelo a cada incidente (Predicción Final)
Mostrando las predicciones del mejor modelo (Random Forest) junto a los datos originales de prueba.

In [ ]:
# Guardar los resultados en el dataframe de prueba para visualizarlos
resultados = X_test.copy()
resultados['Nivel Real (Target)'] = y_test
resultados['Nivel Predicho (RF)'] = rf_predictions

print("Resultados de la asignación del modelo (Predicciones vs Realidad):")
resultados.head(10)